# Выявление закономерностей, определяющих успешность игры

---

## Описание проекта

<p>Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.</p>
<p>Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.</p>
<p>В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».</p>

---

## Описание данных

**Name** — название игры<br>
**Platform** — платформа<br>
**Year_of_Release** — год выпуска<br>
**Genre** — жанр игры<br>
**NA_sales** — продажи в Северной Америке (миллионы проданных копий)<br>
**EU_sales** — продажи в Европе (миллионы проданных копий)<br>
**JP_sales** — продажи в Японии (миллионы проданных копий)<br>
**Other_sales** — продажи в других странах (миллионы проданных копий)<br>
**Critic_Score** — оценка критиков (максимум 100)<br>
**User_Score** — оценка пользователей (максимум 10)<br>
**Rating** — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.<br>

### Примечание
Данные за 2016 год могут быть неполными.

---

### Оглавление

* [1. Открываем файлы с данными и изучаем общую информацию.](#one)
* [2. Предобработка данных.](#two)
* [3. Исследовательский анализ данных.](#three)
* [4. Составляем портрет пользователя каждого региона.](#four)
* [5. Проверка гипотез.](#five)
* [Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.](#six)
* [Средние пользовательские рейтинги жанров Action и Sports разные.](#seven)
* [6. Общие выводы.](#eight)

---

## Шаг 1. Открываем файлы с данными и изучаем общую информацию. <a class="anchor" id="one"></a>

Импортируем необходимые для проекта библиотеки

In [ ]:
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from scipy import stats as st
import seaborn as sns

Откроем файлы с данными. Посмотрим на первые тридцать строк методом `head()`, последние тридцать строк методом `tail()` и изучим столбцы методом `info()`

In [ ]:
# Откроем файл
games = pd.read_csv('datasets/4_games.csv')

In [ ]:
# Посмотрим на первые и последние тридцать строк
display(games.head(30))
display(games.tail(30))

In [ ]:
games.info()

### Выводы

- Мы прочитали данные и преобразовали их в датафрейм games.<br>

- Дата фрейм состоит из 16715 строк и 11 столбцов.

- Наименование столбцов с большой буквы, для удобства переведем их в нижний регистр.

- В столбце `Name` и `Genre` обнаружены 2 пропуска, возможно мы их так и оставим или удалим.

- В столбце `Year_of_Realease` обнаружены пропуски и вероятнее всего мы их удалим.

- Также множество пропущенных значений в столбцах `Critic_Score`, `User_Score` и `Rating` и вероятнее всего мы их оставим.

- В столбце `User_Score` имеются аббревиатуры **tbd**. Разберем ее в следующем шаге.

---

## Шаг 2. Предобработка данных. <a class="anchor" id="two"></a>

Проверим наличие дубликатов в наших таблицах:

In [ ]:
games.duplicated().sum()

Дубликатов не обнаружено

### Заменим названия столбцов
(приведем к нижнему регистру)

In [ ]:
games.columns = games.columns.str.lower()

In [ ]:
games.columns

### Преобразуем данные в нужные типы.
Опишем, в каких столбцах заменили тип данных и почему;

Преобразуем тип данных в столбце `year_of_release` в *datetime year*, предварительно обработав пропущенные значения.
Пропущенные значения могли появиться в результате неправильного ввода или выгрузки, либо просто информация отстуствовала. Посмотрим на эти значения:

In [ ]:
games['platform'].loc[games['year_of_release'].isna()].unique()

К сожалению мы не сможем предугадать год и поэтому отбросим пропущенные значения это около 2,5% от датасета.

In [ ]:
games.dropna(subset = ['year_of_release', 'genre', 'name'], inplace = True)

In [ ]:
games.info()

In [ ]:
games['year_of_release']

In [ ]:
games['year_of_release'] = pd.to_datetime(games['year_of_release'], format='%Y')
games['year_of_release'] = games['year_of_release'].dt.year
games

In [ ]:
games.info()

В столбцах `critic_score`, `user_score` и `rating` множество пропущенных значений и их мы не сможем заполнить, т.к. рейтинг геймеров и критиков предугадать практически невозможно в нашем случае т.к. мало данных. А возраст заполнить также невозможно т.к. нужно знать игры. Поэтому оставим как есть. Пропуски появились ввиду отсутствия информации (видать информация бралась с отного источника на котором не все отзывы есть, в рекомендациях опишем выход для таких ситуаций).

Мы обнаружили аббревиатуру `tbd`, которая означает, что рейтинг в будущем будет проставлен. Она имеет лишь намерение пользователя, но не более того))) Поэтому заменим ее на None и приведем столбцы к типу `float`

In [ ]:
games.loc[games['user_score'] == 'tbd', 'user_score'] = None

In [ ]:
games['user_score'] = games['user_score'].astype(float)

Заменим пропущенные значения в рейтингах на `unknown`

In [ ]:
games['user_score'] = games['user_score'].fillna('unknown')

In [ ]:
games['critic_score'] = games['critic_score'].fillna('unknown')

In [ ]:
games['rating'] = games['rating'].fillna('unknown')

In [ ]:
games.info()

Посмотрим на столбце `rating`, возможно там будут дубли.

In [ ]:
games['rating'].unique()

Посчитаем суммарные продажи во всех регионах и запишем их в отдельный столбец.

In [ ]:
# Суммарные продажи во всех регионах
games['all_sales'] = games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales']

games

### Вывод

- Мы привели названия столбцов к нижнему регистру
- Удалили строки с пропущенными значениями в столбце `year_of_release` и привели к нужному типу данных.
- Избавились от аббревиатуры *tbd*
- Привели значения в столбцах `user_score` к `float`
- Добавили новый столбец `all_sales` - суммарные продажи по всем регионам.

---

## Шаг 3. Исследовательский анализ данных. <a class="anchor" id="three"></a>

### Посмотрим, сколько игр выпускалось в разные годы. И ответим на вопрос: Важны ли данные за все периоды?

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Распределения количества игр по годам')
sns.distplot(games['year_of_release'], kde=False)

In [ ]:
count_by_year = games.pivot_table(index = ['year_of_release'], \
                                  values = 'name', aggfunc = 'count')
count_by_year.columns = ['count']
display(count_by_year)

Для репрезентативности выборки необходимо, чтобы было от 30 значений, поэтому в последующих расчетах возьмем только те года, где количество игр более 30.

In [ ]:
count_by_year.query('count >= 30')

In [ ]:
count_by_year.index

In [ ]:
need_year = count_by_year.index
    
games = games.query('year_of_release in @need_year')

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Распределения количества игр по годам')
sns.distplot(games['year_of_release'], kde=False)

#### Выводы

Мы отобрали только те года, в которые выпустили минимум 30 игр.

---

### Посмотриv, как менялись продажи по платформам. Выберем платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?

Вычислим топ 20 платформ по продажам

In [ ]:
platform_need = games.groupby('platform')['all_sales'].sum().sort_values(ascending = False).head(20)
platform_need

В дальнейшем нам понадобятся данные по платформе `PC`, поэтому эта платформа будет отправной точкой. Возьмем всех до `PC` и похожие по продажам платформы. Из нашего датасета выберем только те, которые входят в топ 14.

In [ ]:
platform_need = platform_need.head(14).index
platform_need = games.query('platform in @platform_need')

Построим графики продаж по каждой платформе. Для дальнейшего определения время жизни платформы примем минимальные продажи в 25 млн. Отметим красной пунктирной линией отметку в 20 млн.

In [ ]:
pd.pivot_table(platform_need,
              index = 'year_of_release',
              columns = 'platform',
              values = 'all_sales',
              aggfunc = 'sum').plot(figsize=(15, 10))
y = 20
plt.axhline(y, linestyle = ':', color = 'red')
plt.title('Графики зависимости продаж от года по каждой платформе')
plt.xlabel('Год')
plt.ylabel('Суммарные продажи в млн $')
plt.legend()

Определим максимальное, среднее и минимальное время жизни платформы при заданном минимальном уровне продаж

In [ ]:
#lifetime = games.query('all_sales >= @y')
lifetime = games.pivot_table(index=['platform', 'year_of_release'], values = 'all_sales', aggfunc='sum')
lifetime = lifetime.query('all_sales >= @y')

In [ ]:
lifetime = lifetime.reset_index()

Максимальное время жизни платформы

In [ ]:
lifetime.groupby('platform')['year_of_release'].count().max()

Среднее время жизни платформы

In [ ]:
lifetime.groupby('platform')['year_of_release'].count().median()

Минимальное время жизни платформы

In [ ]:
lifetime.groupby('platform')['year_of_release'].count().min()

#### Выводы

Мы отобрали топ 14 платформ по продажам. 

- Максимальное время жизни одной платформы - 10 лет;
- Среднее время жизни одной платформы - 6 лет;
- Минимальное время жизни одной платформы - 3 года.

---

Возьмем данные за соответствующий актуальный период. Актуальный период определим самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год.
Данные за предыдущие годы в работе учитывать не будем. Возьмем актуальный период по минимальному жизни платформы.

In [ ]:
games = games.query('year_of_release >= 2013')

### Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.

In [ ]:
pd.pivot_table(games,
              index = 'year_of_release',
              columns = 'platform',
              values = 'all_sales',
              aggfunc = 'sum').plot(figsize=(15, 10), xlim=(2013,2016))
y = 20
plt.axhline(y, linestyle = ':', color = 'red')
plt.title('Графики зависимости продаж от года по каждой платформе')
plt.xlabel('Год')
plt.ylabel('Суммарные продажи в млн $')
plt.legend()

#### Выводы

- Пожалуй, самые перспективные платформы это `PS4` и `XOne`, с учетом того, что данные за 2016 частично отсутствуют они растут с 2013 года.

- Также стоит отметить платформу `PC`, которая самая живучая))) Хоть и продажи не такие высокие.

---

### Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.

In [ ]:
pd.pivot_table(games,
              index = 'year_of_release',
              columns = 'platform',
              values = 'all_sales',
              aggfunc = 'sum').boxplot(figsize=(15,10))

plt.axhline(y, linestyle = ':', color = 'red')
plt.title('График "ящик с усами" по глобальным продажам игр в разбивке по платформам')
plt.xlabel('Платформа')
plt.ylabel('Суммарные продажи в млн $')

Самый малый разброс у платформы `PC` - стабильно низкие показатели)) `PS3`, `XOne` и `X360` имеют похожие доверительные интервалы и и самый большой разброс.

Лидером выступает платформа `PS4`.

---

### Посмотрим, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами. Сформулируем выводы.

In [ ]:
critic_games = games.query('critic_score != "unknown"')

In [ ]:
critic_games['critic_score'] = critic_games['critic_score'].astype(float)

In [ ]:
user_games = games.query('user_score != "unknown"')

In [ ]:
user_games['user_score'] = user_games['user_score'].astype(float)

In [ ]:
critic_games.query('platform == "PS4"').plot(x='critic_score', y='all_sales', kind='scatter', figsize=(15, 5))
plt.title('График зависимости продаж от оценки критиков')
plt.xlabel('Оценка критиков')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
user_games.query('platform == "PS4"').plot(x='user_score', y='all_sales', kind='scatter', figsize=(15, 5))
plt.title('График зависимости продаж от оценки пользователей')
plt.xlabel('Оценка пользователей')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
print(critic_games['critic_score'].corr(critic_games['all_sales']))

In [ ]:
print(user_games['user_score'].corr(user_games['all_sales']))

### Выводы

Рейтинг критиков сильнее кореллирует с продажами, чем рейтинг пользователей.

---

### Соотнесем выводы с продажами игр на других платформах.

Напишем функцию, которая для каждой платформы посчитает коэффициент корреляции и построит график рассеяния.

In [ ]:
for platform in critic_games['platform'].unique():
    critic_games.query('platform == @platform').plot(x='critic_score', y='all_sales', kind='scatter', label=platform)
    plt.title('График зависимости продаж от оценки критиков')
    plt.xlabel('Оценка критиков')
    plt.ylabel('Суммарные продажи в млн $')
    print('Коэффициент корреляции рейтинг критиков с продажами для платформы', platform,\
          critic_games.query('platform == @platform')['critic_score']\
          .corr(critic_games.query('platform == @platform')['all_sales']))
for platform in user_games['platform'].unique():    
    user_games.query('platform == @platform').plot(x='user_score', y='all_sales', kind='scatter', label=platform)
    plt.title('График зависимости продаж от оценки пользователей')
    plt.xlabel('Оценка пользователей')
    plt.ylabel('Суммарные продажи в млн $')
    print('Коэффициент корреляции рейтинг пользователей с продажами для платформы', platform,\
          user_games.query('platform == @platform')['user_score']\
          .corr(user_games.query('platform == @platform')['all_sales']))

### Посмотрим на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

In [ ]:
plt.figure(figsize=(15, 5))
sns.stripplot(x = "genre", y = "all_sales", data = games)
plt.title('График зависимости продаж от жанра')
plt.xlabel('Жанр')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
genre_sales = games.pivot_table(index='genre', values = 'all_sales', aggfunc = 'sum')
genre_sales.sort_values('all_sales', ascending = False)

#### Выводы

- Самые распространненные жанры: `Action`, `Shooter`
- Немного отстают: `Sports`, `Role-Playing` и `Misc`
- Все очень плохо с продажами: `Strategy`, `Puzzle`

---

## Шаг 4. Составляем портрет пользователя каждого региона. <a class="anchor" id="four"></a>

### Определим для пользователя каждого региона (NA, EU, JP):
Самые популярные платформы (топ-5). Опишите различия в долях продаж.

In [ ]:
region_users = games.pivot_table(index = 'platform', values = ['na_sales', 'eu_sales', 'jp_sales'], aggfunc = 'sum')

In [ ]:
region_users

In [ ]:
plt.figure(figsize=(15, 5))
sns.stripplot(x = "platform", y = "na_sales", data = games)
plt.title('График распределения продаж в Северной Америке по платформам')
plt.xlabel('Платформа')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
region_users['na_sales'].sort_values(ascending = False).head()

In [ ]:
plt.figure(figsize=(15, 5))
sns.stripplot(x = "platform", y = "eu_sales", data = games)
plt.title('График распределения продаж в Европе по платформам')
plt.xlabel('Платформа')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
region_users.sort_values('eu_sales', ascending = False).head()

In [ ]:
plt.figure(figsize=(15, 5))
sns.stripplot(x = "platform", y = "jp_sales", data = games)
plt.title('График распределения продаж в Японии по платформам')
plt.xlabel('Платформа')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
region_users.sort_values('jp_sales', ascending = False).head()

In [ ]:
# Зададим сетку для отрисовки графиков
fig, ax = plt.subplots(figsize = (15, 5))

# Зададим имена регионов
regions = ['na', 'jp', 'eu']

for i, region in enumerate(regions):
    column_name = '{}_sales'.format(region) # По шаблону формируем имя столбца, к-й хотим прочитать
    current = games.groupby('platform').agg({column_name: 'sum'}).sort_values(by=column_name, ascending=False).head(7)
    current.plot(y = column_name, kind = 'pie', ax = plt.subplot(1, len(regions), i + 1)) # Строим графики
    plt.title('Влияние платформы в регионе {}'.format(region.upper()))
    plt.legend().remove() # Удаляем индивидуальные легенды
    plt.axis('off')
    
# Строим общую легенду - получаем из рисунка все точки данных
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [], []  # Два пустых массива
for plot in lines_labels:
    for i in range(len(plot[0])):  # В каждом графике перебираем все сектора и имена
        line = plot[0][i]          # Сектор
        label = plot[1][i]         # Имя
        if not label in labels:    # Если имя еще не добавили
            lines += [line]
            labels += [label]
fig.legend(lines, labels)

Как мы видим по топ-5 платформ по каждому региону - месторасположение производителя влияет и на продажи.

---

### Определим для пользователя каждого региона (NA, EU, JP):

Самые популярные жанры (топ-5). Поясним разницу.

In [ ]:
region_genre = games.pivot_table(index = 'genre', values = ['na_sales', 'eu_sales', 'jp_sales'], aggfunc = 'sum')

In [ ]:
plt.figure(figsize=(15, 8))
sns.stripplot(x = "genre", y = "na_sales", data = games)
plt.title('График распределения продаж в Северной Америке по жанрам')
plt.xlabel('Жанр')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
region_genre.sort_values('na_sales', ascending = False).head()

In [ ]:
plt.figure(figsize=(15, 5))
sns.stripplot(x = "genre", y = "eu_sales", data = games)
plt.title('График распределения продаж в Европе по жанрам')
plt.xlabel('Жанр')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
region_genre.sort_values('eu_sales', ascending = False).head()

In [ ]:
plt.figure(figsize=(15, 5))
sns.stripplot(x = "genre", y = "jp_sales", data = games)
plt.title('График распределения продаж в Японии по жанрам')
plt.xlabel('Жанр')
plt.ylabel('Суммарные продажи в млн $')

In [ ]:
region_genre.sort_values('jp_sales', ascending = False).head()

In [ ]:
# Зададим сетку для отрисовки графиков
fig, ax = plt.subplots(figsize = (15, 5))

# Зададим имена регионов
regions = ['na', 'jp', 'eu']

for i, region in enumerate(regions):
    column_name = '{}_sales'.format(region) # По шаблону формируем имя столбца, к-й хотим прочитать
    current = games.groupby('genre').agg({column_name: 'sum'}).sort_values(by=column_name, ascending=False).head(8)
    current.plot(y = column_name, kind = 'pie', ax = plt.subplot(1, len(regions), i + 1)) # Строим графики
    plt.title('Влияние жанра в регионе {}'.format(region.upper()))
    plt.legend().remove() # Удаляем индивидуальные легенды
    plt.axis('off')
    
# Строим общую легенду - получаем из рисунка все точки данных
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [], []  # Два пустых массива
for plot in lines_labels:
    for i in range(len(plot[0])):  # В каждом графике перебираем все сектора и имена
        line = plot[0][i]          # Сектор
        label = plot[1][i]         # Имя
        if not label in labels:    # Если имя еще не добавили
            lines += [line]
            labels += [label]
fig.legend(lines, labels)

В Европе и Америке очень похожие показатели и лидерами выступают:
- `Action`
- `Shooter`
- `Sports`

В Японии же лидером оказывается `Role-Playing`


Вероятно на выбор игры влияет менталитет и культура населения и тут Европа совпала с Америкой, чего не скажешь о Японии.

---

### Определим для пользователя каждого региона (NA, EU, JP):

Влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [ ]:
rating_sales = games.pivot_table(index = 'rating', values = ['na_sales', 'eu_sales', 'jp_sales'], aggfunc = 'sum')

In [ ]:
rating_sales.sort_values('na_sales', ascending = False).head()

In [ ]:
rating_sales.sort_values('eu_sales', ascending = False).head()

In [ ]:
rating_sales.sort_values('jp_sales', ascending = False).head()

In [ ]:
# Зададим сетку для отрисовки графиков
fig, ax = plt.subplots(figsize = (15, 8))

# Зададим имена регионов
regions = ['na', 'jp', 'eu']

for i, region in enumerate(regions):
    column_name = '{}_sales'.format(region) # По шаблону формируем имя столбца, к-й хотим прочитать
    current = games.groupby('rating').agg({column_name: 'sum'})
    current.plot(y = column_name, kind = 'pie', ax = plt.subplot(1, len(regions), i + 1)) # Строим графики
    plt.title('Влияние рейтинга ESRB в регионе {}'.format(region.upper()))
    plt.legend().remove() # Удаляем индивидуальные легенды
    plt.axis('off')
    
# Строим общую легенду - получаем из рисунка все точки данных
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [], []  # Два пустых массива
for plot in lines_labels:
    for i in range(len(plot[0])):  # В каждом графике перебираем все сектора и имена
        line = plot[0][i]          # Сектор
        label = plot[1][i]         # Имя
        if not label in labels:    # Если имя еще не добавили
            lines += [line]
            labels += [label]
fig.legend(lines, labels)

#### Выводы

Самые продаваемые ESRB:
- `M`
- `E`
- `T`

Игры без рейтинга в Японии занимают бОльшую часть, т.к. ESRB - это северо американская организация и, вероятно, японским играм рейтинг она ставит редко.

### Описание типичного геймера каждого региона

| Параметр | Северная Америка | Европа | Япония |
| :-: | :-: | :-: | :-: |
| Платформа |
| 1 | PS4 | PS4 | 3DS |
| 2 | XOne | PS3 | PS3 |
| 3 | X360 | XOne | PSV |
| Жанр |
| 1 | Action | Action | Role-Playing |
| 2 | Shooter | Shooter | Action |
| 3 | Sports | Sports | Misc |
| Рейтинг ESRB |
| 1 | M | M | unknown |
| 2 | unknown | E | T |
| 3 | E | unknown | E |

In [ ]:
# Зададим сетку для отрисовки графиков
fig, ax = plt.subplots(figsize = (15, 8))

# Зададим имена регионов
regions = ['na', 'jp', 'eu']

for i, region in enumerate(regions):
    column_name = '{}_sales'.format(region) # По шаблону формируем имя столбца, к-й хотим прочитать
    current = games.groupby('rating').agg({column_name: 'sum'})
    current.plot(y = column_name, kind = 'pie', ax = plt.subplot(1, len(regions), i + 1), autopct='%1.1f%%') # Строим графики
    plt.title('Влияние рейтинга ESRB в регионе {}'.format(region.upper()))
    plt.legend().remove() # Удаляем индивидуальные легенды
    plt.axis('off')
    
# Строим общую легенду - получаем из рисунка все точки данных
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [], []  # Два пустых массива
for plot in lines_labels:
    for i in range(len(plot[0])):  # В каждом графике перебираем все сектора и имена
        line = plot[0][i]          # Сектор
        label = plot[1][i]         # Имя
        if not label in labels:    # Если имя еще не добавили
            lines += [line]
            labels += [label]
fig.legend(lines, labels)

---

## Шаг 5. Проверка гипотез <a class="anchor" id="five"></a>

### Средние пользовательские рейтинги платформ Xbox One и PC одинаковые. <a class="anchor" id="six"></a>

H0: Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.
    
H1: Средние пользовательские рейтинги платформ Xbox One и PC отличаются.

Выполним выборку по платформе `XOne` и возьмем ненулевой рейтинг

In [ ]:
xbox_one = games.query('platform == "XOne" & user_score != "unknown"')
xbox_one

Выполним выборку по платформе `PC` и возьмем ненулевой рейтинг

In [ ]:
pc = games.query('platform == "PC" & user_score != "unknown"')
pc

Построим графики распределения среднего пользовательского рейтинга по выбранным платформам

In [ ]:
plt.figure(figsize=(15, 5))
(xbox_one['user_score'].hist(bins=10, alpha = 0.5, label = 'XBox One'))
plt.title('График распределения среднего пользовательского рейтинга по выбранным платформам')
plt.legend()
(pc['user_score'].hist(bins=10, alpha = 0.5, label = 'PC'))
plt.title('График распределения среднего пользовательского рейтинга по выбранным платформам')
plt.xlabel('Пользовательский рейтинг')
plt.legend()

В данном тесте мы используем статистический критерий Стьюдента, потому что количество измерений больше 150 и выборочное среднее распределено нормально.

Проверим отличаются ли дисперсии более чем на 5%.

In [ ]:
xbox_one['user_score'].var()

In [ ]:
pc['user_score'].var()

Дисперсии отличаются более чем на 5%.

Применим ттест для проверки нулевой гипотезы. Значения альфа возьмем 0.05.

In [ ]:
xbox_vs_pc = st.ttest_ind(
    xbox_one['user_score'],
    pc['user_score'],
    equal_var=False)

alpha = 0.05

In [ ]:
print('p-значение:', xbox_vs_pc.pvalue)

if xbox_vs_pc.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

#### Вывод

- Мы приняли нулевую гипотезу о равенстве среднего пользовательского рейтинга в **XBox One** и **PC**.

- По графику видим, что средний пользовательский рейтинг одинаков.

---

### Средние пользовательские рейтинги жанров Action и Sports одинаковые. <a class="anchor" id="seven"></a>

H0: Средние пользовательские рейтинги жанров Action и Sports одинаковые.
    
H1: Средние пользовательские рейтинги жанров Action и Sports отличаются.

Выполним выборку по жанру `Action` и возьмем ненулевой рейтинг

In [ ]:
action = games.query('genre == "Action" & user_score != "unknown"')
action

Выполним выборку по жанру `Sports` и возьмем ненулевой рейтинг

In [ ]:
sports = games.query('genre == "Sports" & user_score != "unknown"')
sports

Построим графики распределения среднего пользовательского рейтинга по выбранным жанрам

In [ ]:
plt.figure(figsize=(15, 5))
(action['user_score'].hist(bins=10, alpha = 0.5, label = 'Action'))
plt.title('График распределения среднего пользовательского рейтинга по выбранным жанрам')
plt.legend()
(sports['user_score'].hist(bins=10, alpha = 0.5, label = 'Sports'))
plt.title('График распределения среднего пользовательского рейтинга по выбранным жанрам')
plt.xlabel('Пользовательский рейтинг')
plt.legend()

В данном тесте мы используем статистический критерий Стьюдента, потому что количество измерений больше 150 и выборочное среднее распределено нормально.

Проверим отличаются ли дисперсии более чем на 5%.

In [ ]:
action['user_score'].var()

In [ ]:
sports['user_score'].var()

Дисперсии отличаются более чем на 5%.

Применим ттест для проверки нулевой гипотезы. Значения альфа возьмем 0.05.

In [ ]:
action_vs_sports = st.ttest_ind(
    action['user_score'],
    sports['user_score'],
    equal_var=False)

alpha = 0.05

In [ ]:
print('p-значение:', action_vs_sports.pvalue)

if action_vs_sports.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

#### Вывод

- Мы отвергли нулевую гипотезу о равенстве среднего пользовательского рейтинга в жанрах **Action** и **Sports** и приняли альтернативную гипотезу - средний рейтинг не равен.

- По графику видим, что средний рейтинг в жанре **Action** выше.

---

## Шаг 6. Общие выводы <a class="anchor" id="eight"></a>

- Мы прочитали данные и преобразовали их в датафрейм games.<br>

- Дата фрейм состоит из 16715 строк и 11 столбцов.

- Мы предобработали данные и добавили новый столбец `all_sales` - суммарные продажи по всем регионам.

- Мы отобрали топ 14 платформ по продажам. Среднее время жизни одной платформы - 6 лет.

- Рейтинг критиков сильнее кореллирует с продажами, чем рейтинг пользователей.

- Сделали вывод, что самая перспективная платформа это `PS4` и `PC`, т.к. она актуальна на протяжении всего времени.

- Мы составили портрет пользователя для каждого региона

| Параметр | Северная Америка | Европа | Япония |
| :-: | :-: | :-: | :-: |
| Платформа |
| 1 | PS4 | PS4 | 3DS |
| 2 | XOne | PS3 | PS3 |
| 3 | X360 | XOne | PSV |
| Жанр |
| 1 | Action | Action | Role-Playing |
| 2 | Shooter | Shooter | Action |
| 3 | Sports | Sports | Misc |
| Рейтинг ESRB |
| 1 | M | M | unknown |
| 2 | unknown | E | T |
| 3 | E | unknown | E |

- Мы приняли нулевую гипотезу о равенстве среднего пользовательского рейтинга в **XBox One** и **PC**.

- По графику видим, что средний пользовательский рейтинг одинаков.

- Мы отвергли нулевую гипотезу о равенстве среднего пользовательского рейтинга в жанрах **Action** и **Sports** и приняли альтернативную гипотезу - средний рейтинг не равен.

- По графику видим, что средний рейтинг в жанре **Action** выше.

### Рекомендации

- Для полной картины отзывов необходимо выполнить скрейпинг отзывов с различных игровых площадок.
- Для планирования продаж на 2017-ой год рекомендуем воспользоваться общими выводами.